**Date: 09.09.2022**
<br>
**Author: Mohammad Suhash Jahan**


This script is written for the data visualisation, performance metrics and energy consumption calculation of the mMCDI plant of the project 'innovatION' (https://www.innovat-ion.de) as part of my master's thesis titled 'Selective desalination by Membrane Capacitive Deionisation'. The script is written using Google Colaboratory (or popularly known as Colab). But it is also possible to use other Python IDEs.
<br>
The script contains several functions for performing the following tasks:
1. Preprocessing of the dataset obtained from the plant.
2. Visulaisation of the data and divide the data into cycles.
3. Finally, the desalination performance and energy consumption is calculated.





In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Loading data
#name = pd.read_csv('Enter your file location)

Functions:

1) preprocessing: This function does three things. It renames the columns name into understandable keywords, calculates the timesteps of the logged data and finds the average voltage of the cells. Input of the function is the dataframe.

In [3]:
#preprocessing
def preprocessing(df):
  df.rename(columns = {'Loc Time':'time','M2':'flow_rate','M4':'feed_ph', 'M5':'feed_cond','M6':'feed_temp','M8':'eff_ph','M9':'eff_cond','M10':'eff_temp','M11':'voltage','M12':'current','M13':'Z1V1','M14':'Z1V2','M15':'Z1V3'},inplace = True)
  df['time']=pd.to_datetime(df.time,format='%Y-%m-%d %H:%M:%S.%f')
  df['time_diff'] = df["time"].diff().apply(lambda x: x/np.timedelta64(1, 's')).fillna(0).astype('float')#calculating time difference in s
  df['timestep']=df['time_diff'].cumsum()
  df['avg_voltage'] = df[['Z1V1','Z1V2','Z1V3']].mean(axis=1)
  return df.head()

2) plot_variables: It plots current, average cell voltage, effluent conductivity and effluent pH over time. It is helpful for observing the changes of the variables over time. It takes the dataframe as input.

In [4]:
#plot variables
def plot_variables(df):
  fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
  fig.tight_layout()
  ax1, ax2, ax3, ax4 = axes.flatten()
  ax1.plot(df.timestep, df.current, color='0', ls='solid')
  ax1.axhline(y=0, color='b', linestyle='-',label='X-axis')
  ax1.set_xlabel('Time (s)')
  ax1.set_ylabel('Current (A)')
  ax1.grid(True)
  ax2.plot(df.timestep, df.avg_voltage, color='r', ls='solid')
  ax2.axhline(y=0, color='b', linestyle='-',label='X-axis')
  ax2.set_xlabel('Time (s)')
  ax2.set_ylabel('Voltage (V)')
  ax2.grid(True)
  ax3.plot(df.timestep, df.eff_cond, color='b', ls='solid')
  ax3.set_xlabel('Time (s)')
  ax3.set_ylabel('Conductivity (µS/cm)')
  ax3.grid(True)
  ax4.plot(df.timestep, df.eff_ph, color='g', ls='solid')
  ax4.set_xlabel('Time (s)')
  ax4.set_ylabel('pH')
  ax4.grid(True)

3) divide_cycles: This function divides the data according to each half-cycle (i.e., each adsorption/desorption cycle). It gives the row index of each cycle's end time, which can be used to determine the duration of one steady-state adsorption cycle. Here, the function takes the 4th cycle as steady cycle. The dataframe is the input of the function.

In [5]:
#divide data into cycles
def divide_cycles(df):
  df['current_multi']=df['current']*df.shift(1)['current'] #multiplying consecutive currents to find change in cycle
  end_cycle=df.index[df['current_multi']<0].tolist()        
  number_of_cycles=len(end_cycle)/2
  start=end_cycle[5] #4th cycle
  stop=end_cycle[6]
  adsorption_time=df.loc[stop,'timestep']-df.loc[start,'timestep']
  tad="{:.2f}".format(adsorption_time)
  print('End index of half cycles: ',end_cycle)
  print('Number of total cycles: ',number_of_cycles)
  print('Starting index of 4th adsorption cycle: ',start)
  print('Stopping index of 4th adsorption cycle: ',stop)
  print('Adsorption duration:',tad,'s') 

4) charge_efficiency: This function takes the dataset and the regression coefficient of the conductivity-concentration relationship of the salt as input and prints the charge efficieny.

In [6]:
#charge efficiency
def charge_efficiency(df,reg_coef):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  df['diff_c'] = df['feed_cond']-df['eff_cond'] #calculating concentration difference of effluent and feed
  F = 96485/(3600*58*1000) 
  sum_diff_c = df.iloc[start:stop].diff_c.sum() #isolating one steady-state cycle
  sum_i = df.iloc[start:stop].current.sum()
  deltac = sum_diff_c
  delt_conc = deltac/reg_coef #converting conductivity to concentration
  Q = df.iloc[start:stop].flow_rate.mean()
  ce = ((F*Q*delt_conc)/sum_i)*100
  ce = "{:.0f}".format(ce)
  print('Charge Efficiency: ', ce,'%')

5) salt_retention: It takes the dataframe as input and return the percentage of salt retention.

In [7]:
#salt retention
def salt_retention(df):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  df['diff_c'] = df['feed_cond']-df['eff_cond']
  sum_diff_c = df.iloc[start:stop].diff_c.sum()
  sum_feed_c = df.iloc[start:stop].feed_cond.sum()
  sr = (sum_diff_c/sum_feed_c)*100
  sr = "{:.2f}".format(sr)
  print('Salt Retention: ', sr,'%')

6) sac_asar: This function takes the dataframe, mass of the electrode and the regression coefficient of the conductivity-concentration relationship of the salt as input and return the salt adsorption capacity and average salt adsorption rate.

In [8]:
#salt adsorption capacity & average salt adsorption rate
def sac_asar(df,m,reg_coef):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  tad=df.loc[stop,'timestep']-df.loc[start,'timestep']
  df['diff_c'] = df['feed_cond']-df['eff_cond']
  sum_diff_c = df.iloc[start:stop].diff_c.sum()
  deltac = sum_diff_c
  delt_conc = deltac/reg_coef
  Q = df.iloc[start:stop].flow_rate.mean()
  sac = (Q*delt_conc)/(m*3600)
  sac_2f = "{:.2f}".format(sac)
  asar=sac/(tad/60)
  asar = "{:.2f}".format(asar)
  print('SAC: ',sac_2f,'mg/g')
  print('ASAR: ',asar,'mg/g/min')

7) sec: It calculates the specific energy consumption of the desalination process for per mg of salt removed and takes the dataframe and regression coefficient of the conductivity-concentration relationship of the salt as input.

In [9]:
#specific energy consumption
def sec(df,reg_coef):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  df['v_i']=df['avg_voltage']*df['current']
  sum_vi = df.iloc[start:stop].v_i.sum()
  df['diff_c'] = df['feed_cond']-df['eff_cond']
  sum_diff_c = df.iloc[start:stop].diff_c.sum()
  delt_conc = sum_diff_c/reg_coef
  Q = df.iloc[start:stop].flow_rate.mean()
  sec = (3600*sum_vi)/(Q*delt_conc)
  sec = "{:.2f}".format(sec)
  print('SEC: ',sec,'J/mg')

8) sec_kwhm3: It takes the dataframe as input and calculate the enery requirement to desalinate per m^3 of water for RVD operation.

In [10]:
#specific energy consumption in kWh/m^3
def sec_kwhm3(df):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  de_stop=end_cycle[7]
  df['v_i']=df['avg_voltage']*df['current']
  sum_vi_ad = df.iloc[start:stop].v_i.sum()
  sum_vi_de= df.iloc[stop:de_stop].v_i.sum()
  Q = df.iloc[start:stop].flow_rate.mean()
  tad=df.loc[stop,'timestep']-df.loc[start,'timestep']
  tde=df.loc[de_stop,'timestep']-df.loc[stop,'timestep']
  sec_kwhm3=(sum_vi_ad+sum_vi_de)/(Q*tad)
  sec_kwhm3 = "{:.2f}".format(sec_kwhm3)
  print('SEC: ',sec_kwhm3,'kWh/m^3')

9) throughput: This function calculates the thorughput and takes three inputs - the dataframe, number of electrodes and area of an electrode.

In [11]:
#throughput
def throughput(df,n,A):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  tad=df.loc[stop,'timestep']-df.loc[start,'timestep']
  V_d =  df.iloc[start:stop].flow_rate.mean()*(tad/3600)
  t_cycle = tad/3600
  p = V_d/(n*A*t_cycle)
  p= "{:.2f}".format(p)
  print('Throughput:',p,'L/h/m^2')

10) water_recovery: This function takes the dataframe as input and provides water recovery in percentage as output. This function calculates correctly when the flowrate was same in adsorption and desorption cycle.

In [12]:
#water recovery
def water_recovery(df):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start_ad=end_cycle[5]
  stop_ad=end_cycle[6]
  stop_de=end_cycle[7]
  adsorption_time=df.loc[stop_ad,'timestep']-df.loc[start_ad,'timestep']
  desorption_time=df.loc[stop_de,'timestep']-df.loc[stop_ad,'timestep']
  wr=(adsorption_time/(adsorption_time+desorption_time))*100
  wr= "{:.2f}".format(wr)
  print('Water Recovery: ',wr,'%')

11) sec_kwhm3_zvd: It takes the dataframe as input. If the experiment was run in ZVD mode than it calculates the energy requirements by considering the adsorption cycle only.

In [13]:
#specific energy consumption in kWh/m^3 for zvd
def sec_kwhm3_zvd(df):
  df['current_multi']=df['current']*df.shift(1)['current']
  end_cycle=df.index[df['current_multi']<0].tolist()
  start=end_cycle[5]
  stop=end_cycle[6]
  df['v_i']=df['avg_voltage']*df['current']
  sum_vi_ad = df.iloc[start:stop].v_i.sum()
  Q = df.iloc[start:stop].flow_rate.mean()
  tad=df.loc[stop,'timestep']-df.loc[start,'timestep']
  sec_kwhm3=(sum_vi_ad)/(Q*tad)
  sec_kwhm3 = "{:.2f}".format(sec_kwhm3)
  print('SEC: ',sec_kwhm3,'kWh/m^3')